In [ ]:
!pip install matplotlib --upgrade
import numpy as np
import matplotlib.pyplot as plt
import abc
from math import sqrt

In [ ]:
# define function classes

class Obj:
  @abc.abstractmethod
  def fval(self, x):
    pass

  @abc.abstractmethod
  def grad(self, x):
    pass

class QuadObj(Obj):
  def fval(self, x):
    return 0.5 * x.T @ self.A @ x - self.b @ x

  def grad(self, x):
    return self.A @ x - self.b

  def ferr(self, x):
    return self.fval(x) - self.fval(self.x_opt)

class TwoBandQuadObj(QuadObj):
  def __init__(self, mu1, L1, mu2, L2, dim):
    self.mu1 = mu1
    self.mu2 = mu2
    self.L1 = L1
    self.L2 = L2
    self.dim = dim

    _Lambd = np.diag(np.concatenate([np.linspace(mu1, L1, dim//2),
                  np.linspace(mu2, L2, dim//2)]))
    _Q, _ = np.linalg.qr(np.random.randn(dim,dim))

    self.A = _Q @ _Lambd @ _Q.T
    self.x_opt = np.random.randn(dim)
    self.b = self.A @ self.x_opt


def gd_scheduled(x_init, obj, L_lst):
    x = x_init
    ferr_lst = []
    for L in L_lst:
        x = x - 1/L * obj.grad(x)
        ferr_lst.append(obj.ferr(x))
    return ferr_lst

## Recursive BSLS is stable

In [ ]:
mu1=1e-3; L1=2e-3; mu2=5e-1; L2=1; dim=128; 
obj = TwoBandQuadObj(mu1=mu1, L1=L1, mu2=mu2, L2=L2, dim=dim)

x_init = np.random.randn(dim)

good_L_lst = ([L1] * 1 + [L2] * 12) * 20
good_ferr_lst = gd_scheduled(x_init, obj, good_L_lst)

bad1_L_lst = ([L2] * 240 + [L1] * 20) * 1
bad1_ferr_lst = gd_scheduled(x_init, obj, bad1_L_lst)

bad2_L_lst = ([L1] * 20 + [L2] * 240) * 1
bad2_ferr_lst = gd_scheduled(x_init, obj, bad2_L_lst)

# ax = plt.gca()
# # ax.semilogy(good_ferr_lst)
# ax.semilogy(bad_ferr_lst)
# for step_size in 10 ** np.linspace(-3, 0, 10):
#   const_ferr_lst = gd_scheduled(x_init, obj, [1.0/step_size] * 220)
#   ax.semilogy(const_ferr_lst, label=step_size)
# plt.semilogy(good_ferr_lst)

_, axes = plt.subplots(2,3, figsize=(12,4.5), gridspec_kw={'height_ratios': [3, 1]}, sharex=True)

axes[0,0].semilogy(good_ferr_lst)
axes[0,0].set_ylabel('functional suboptimality', size='large')
axes[1,0].semilogy(1/np.array(good_L_lst), ',')
axes[1,0].set_ylabel('step size', size='large')


axes[0,1].semilogy(bad1_ferr_lst)
# axes[0,1].set_ylabel('function suboptimality')
axes[1,1].semilogy(1/np.array(bad1_L_lst), ',')

# axes[1,1].set_ylabel('step size')

axes[0,2].semilogy(bad2_ferr_lst)
# axes[0,1].set_ylabel('function suboptimality')
axes[1,2].semilogy(1/np.array(bad2_L_lst), ',')
# axes[1,1].set_ylabel('step size')

axes[0,0].set_ylim([1e-15, 1e7])
axes[0,1].set_ylim([1e-15, 1e7])
axes[0,2].set_ylim([1e-15, 1e21])

for j in range(0, 3):
  axes[1, j].set_xlabel('iterations', size='large')
  axes[1, j].set_yticks([1,  1e3])
  axes[1, j].set_ylim([0.5, 2e3])

axes[0,0].set_title('Interlacing order via recursive BSLS')
axes[0,1].set_title('Bad order I - small steps first')
axes[0,2].set_title('Bad order II - big steps first')

plt.tight_layout(h_pad=0.1, w_pad=0.1)
plt.savefig('why_order_matters.pdf')

In [ ]:
mu1=1e-3; L1=2e-3; mu2=5e-1; L2=1; dim=128; 
obj = TwoBandQuadObj(mu1=mu1, L1=L1, mu2=mu2, L2=L2, dim=dim)

x_init = np.random.randn(dim)

good_L_lst = ([L1] * 1 + [L2] * 12) * 20
good_ferr_lst = gd_scheduled(x_init, obj, good_L_lst)

bad1_L_lst = ([L2] * 240 + [L1] * 20) * 1
bad1_ferr_lst = gd_scheduled(x_init, obj, bad1_L_lst)

bad2_L_lst = ([L1] * 20 + [L2] * 240) * 1
bad2_ferr_lst = gd_scheduled(x_init, obj, bad2_L_lst)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(4,4))
fig.subplots_adjust(hspace=0.05)  # adjust space between axes

# plot the same data on both axes
ax1.semilogy(good_ferr_lst, linestyle='solid', color='C0', label='Interlacing order - BSLS')
ax2.semilogy(good_ferr_lst, linestyle='solid', color='C0', label='Interlacing order - BSLS')

ax1.semilogy(bad1_ferr_lst, linestyle='dotted', color='C1', label='Bad order I - small steps first')
ax2.semilogy(bad1_ferr_lst, linestyle='dotted', color='C1', label='Bad order I - small steps first')

ax1.semilogy(bad2_ferr_lst, linestyle='dashed', color='C2', label='Bad order II - big steps first')
ax2.semilogy(bad2_ferr_lst, linestyle='dashed', color='C2', label='Bad order II - big steps first')

# zoom-in / limit the view to different portions of the data
ax1.set_ylim(1e10, 1e100)  # outliers only
ax2.set_ylim(1e-14, 1)  # most of the data

# hide the spines between ax and ax2
ax1.spines.bottom.set_visible(False)
ax2.spines.top.set_visible(False)
ax1.xaxis.tick_top()
ax1.tick_params(labeltop=False)  # don't put tick labels at the top
ax2.xaxis.tick_bottom()

# Now, let's turn towards the cut-out slanted lines.
# We create line objects in axes coordinates, in which (0,0), (0,1),
# (1,0), and (1,1) are the four corners of the axes.
# The slanted lines themselves are markers at those locations, such that the
# lines keep their angle and position, independent of the axes size or scale
# Finally, we need to disable clipping.

d = .5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)

# ax2.legend(location=)

ax1.legend(bbox_to_anchor=(0.48, 0.2), loc='center', borderaxespad=0.)

# ax.semilogy(good_ferr_lst, linestyle='solid')
# ax.semilogy(bad1_ferr_lst, linestyle='dotted')
# ax.semilogy(bad2_ferr_lst, linestyle='dashed')

ax2.set_xlabel('# gradient queries', size='large')
ax1.set_ylabel('function error', size='large')
ax2.set_ylabel('function error', size='large')

# fig.subplots_adjust(bottom=0.4)
plt.savefig('why_order_matters.pdf',bbox_inches='tight')